In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
from wordcloud import WordCloud
import scipy.stats as stats
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from math import sqrt
import xgboost as xgb
from xgboost import XGBRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#First, I imported pandas as pd to ensure the library is uploaded so that I can use it's functions. Since I am using Google Colab, I need to mount my google drive
#so that I have access to all my files. I uploaded the data file to my Google Drive.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(f"/content/drive/MyDrive/COPY MBTI complete 4500 (1).csv", encoding='utf-8', encoding_errors='ignore')
df.head()

,tconst,titletype,primarytitle,originaltitle,startyear,runtimeminutes,genres,averagerating,numvotes,lead_character,lead_MBTI,lead_votes,second_lead,secondlead_MBTI,secondlead_votes,relationship
0,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,1994,142.0,Drama,9.3,2934011,Andy Dufresne,INTJ,5340,Warden Norton,ESTJ,3807,Enemy
1,tt0468569,movie,The Dark Knight,The Dark Knight,2008,152.0,"Action,Crime,Drama",9.0,2914377,Bruce Wayne,INTJ,4893,The Joker,ENTP,5942,Enemy
2,tt1375666,movie,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,2587898,Dominick Cobb,ISTP,4034,Arthur,ISTJ,2243,Friend
3,tt0137523,movie,Fight Club,Fight Club,1999,139.0,Drama,8.8,2365490,The Narrator,ISFJ,139,Tyler Durden,ENTP,2834,Rival
4,tt0109830,movie,Forrest Gump,Forrest Gump,1994,142.0,"Drama,Romance",8.8,2294581,Forrest Gump,ISFJ,3720,Jenny Curran,ESFP,3037,Romantic Interest


In [ ]:
lead_mbti_traits = ['lead_I', 'lead_E', 'lead_N', 'lead_S', 'lead_T', 'lead_F', 'lead_J', 'lead_P']

# Initialize the new columns with zeros in the original dataframe
for trait in lead_mbti_traits:
    df[trait] = 0

# Fill the columns with 1 or 0 based on the MBTI types in 'lead_MBTI'
for index, mbti in df['lead_MBTI'].items():
    if 'I' in mbti: df.at[index, 'lead_I'] = 1
    if 'E' in mbti: df.at[index, 'lead_E'] = 1
    if 'N' in mbti: df.at[index, 'lead_N'] = 1
    if 'S' in mbti: df.at[index, 'lead_S'] = 1
    if 'T' in mbti: df.at[index, 'lead_T'] = 1
    if 'F' in mbti: df.at[index, 'lead_F'] = 1
    if 'J' in mbti: df.at[index, 'lead_J'] = 1
    if 'P' in mbti: df.at[index, 'lead_P'] = 1

second_lead_mbti_traits = ['secondlead_I', 'secondlead_E', 'secondlead_N', 'secondlead_S', 'secondlead_T', 'secondlead_F', 'secondlead_J', 'secondlead_P']

# Initialize the new columns with zeros in the original dataframe
for trait in second_lead_mbti_traits:
    df[trait] = 0

# Fill the columns with 1 or 0 based on the MBTI types in 'lead_MBTI'
for index, mbti in df['secondlead_MBTI'].items():
    if 'I' in mbti: df.at[index, 'secondlead_I'] = 1
    if 'E' in mbti: df.at[index, 'secondlead_E'] = 1
    if 'N' in mbti: df.at[index, 'secondlead_N'] = 1
    if 'S' in mbti: df.at[index, 'secondlead_S'] = 1
    if 'T' in mbti: df.at[index, 'secondlead_T'] = 1
    if 'F' in mbti: df.at[index, 'secondlead_F'] = 1
    if 'J' in mbti: df.at[index, 'secondlead_J'] = 1
    if 'P' in mbti: df.at[index, 'secondlead_P'] = 1

In [ ]:
lead_mbti_traits = ['lead_I', 'lead_E', 'lead_N', 'lead_S', 'lead_T', 'lead_F', 'lead_J', 'lead_P']
second_lead_mbti_traits = ['secondlead_I', 'secondlead_E', 'secondlead_N', 'secondlead_S', 'secondlead_T', 'secondlead_F', 'secondlead_J', 'secondlead_P']

# Create a new column representing MBTI pairs
df['mbti_pairs'] = df['lead_MBTI'] + '-' + df['secondlead_MBTI']

# Drop rows with missing values
df = df.dropna()

# Define Feature Set (X) and Target (y)
# Add additional features: mbti_pairs, lead_votes, and secondlead_votes
additional_features = ['lead_votes', 'secondlead_votes']
X = df[lead_mbti_traits + second_lead_mbti_traits + additional_features]

# Manually One-Hot Encode the 'mbti_pairs' column
X_encoded = pd.get_dummies(df['mbti_pairs'], prefix='mbti_pairs')
X1_encoded = pd.get_dummies(df['genres'], prefix='genres')

# Combine one-hot encoded columns with original features
X = pd.concat([X, X_encoded, X1_encoded], axis=1)

# Define the target variable (numvotes)
y = df['numvotes']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = rf_model.predict(X_test)

# Model Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Random Forest Mean Squared Error: {mse}")
print(f"Random Forest R-squared: {r2}")

# Feature Importance
importance = rf_model.feature_importances_

# Create a DataFrame for feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)
pd.set_option('display.max_rows', None)

print("\nFeature Importance:\n", feature_importance)

Random Forest Mean Squared Error: 6037438651.425084
Random Forest R-squared: 0.8843052154768808

Feature Importance:
                                     Feature    Importance
16                               lead_votes  7.100978e-01
17                         secondlead_votes  2.162935e-01
286          genres_Action,Adventure,Sci-Fi  3.251675e-03
250                    mbti_pairs_ISTJ-INFP  2.932347e-03
212                    mbti_pairs_ISFJ-ENTP  2.884847e-03
587            genres_Drama,Sci-Fi,Thriller  1.546435e-03
231                    mbti_pairs_ISFP-ESTJ  1.527535e-03
274                 genres_Action,Adventure  1.469062e-03
387         genres_Adventure,Family,Fantasy  1.461842e-03
279           genres_Action,Adventure,Drama  1.455505e-03
0                                    lead_I  1.380075e-03
1                                    lead_E  1.279884e-03
344                    genres_Action,Sci-Fi  1.249219e-03
260                    mbti_pairs_ISTP-ENTP  1.221949e-03
227         

In [ ]:
# Train the XGBoost Regressor
xgb_model = XGBRegressor(n_estimators=200, max_depth=10, random_state=42)
xgb_model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = xgb_model.predict(X_test)

# Model Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = sqrt(mse)

# Print the results
print(f"XGBoost Root Mean Squared Error: {rmse}")
print(f"XGBoost R-squared: {r2}")

# Feature Importance
importance = xgb_model.feature_importances_

# Create a DataFrame for feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)
pd.set_option('display.max_rows', None)

print("\nFeature Importance:\n", feature_importance)

XGBoost Root Mean Squared Error: 83243.96950483216
XGBoost R-squared: 0.8672096133232117

Feature Importance:
                                     Feature    Importance
16                               lead_votes  6.022761e-02
172                    mbti_pairs_INFP-INTJ  3.075946e-02
68                     mbti_pairs_ENTP-ENTJ  2.732815e-02
227                    mbti_pairs_ISFP-ENTJ  2.625843e-02
231                    mbti_pairs_ISFP-ESTJ  2.530075e-02
17                         secondlead_votes  2.424729e-02
250                    mbti_pairs_ISTJ-INFP  2.003836e-02
54                     mbti_pairs_ENTJ-ESFJ  1.858317e-02
202                    mbti_pairs_INTP-INFP  1.827274e-02
260                    mbti_pairs_ISTP-ENTP  1.745632e-02
75                     mbti_pairs_ENTP-INFP  1.650298e-02
25                     mbti_pairs_ENFJ-ESTP  1.399547e-02
274                 genres_Action,Adventure  1.318489e-02
225                    mbti_pairs_ISFP-ENFJ  1.244784e-02
36                 